### Distilling DeepSeek Coder 1.3B for the purpose of creating a student model for test case assertion generation

First we install and import the needed requirements:

In [23]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [67]:
!pip3 install transformers==4.26.1

  Obtaining dependency information for transformers==4.26.1 from https://files.pythonhosted.org/packages/1e/e2/60c3f4691b16d126ee9cfe28f598b13c424b60350ab339aba81aef054b8f/transformers-4.26.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/100.3 kB ? eta -:--:--
     -------------------------------------- 100.3/100.3 kB 5.6 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers!=0.11.3,<0.14,>=0.11.1 from https://files.pythonhosted.org/packages/62/41/93d3135ec30f596a71490ce11a73572190fe80e85a2aea18f116a520cc41/tokenizers-0.13.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/6.3 MB 6.5 MB/s eta 0:00:01
   --- ------------------------------------ 0.6/6.3 MB 6.1 MB/s eta 0:00:01
   ----- ---------------------------------- 0.9/6.3 MB 6.2 MB/s eta 0:00:01
   ------ --------------------------------- 1.0/6.3 MB 6.6 MB/s eta 0:00:01
   ------ --------

In [1]:
import json
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import (
    T5ForConditionalGeneration,
    RobertaTokenizer,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
import os
import time
from tqdm import tqdm

from Data.student_dataset import StudentDataset

Testing decompression of logits...
Entry loaded successfully.

Teacher prediction:
assertNotNull(ret);
assertEquals(Integer.valueOf(3), ret);

Reference assertions:
assertNotNull(ret);
assertEquals(Long.class, ret.getClass());
assertEquals(123L, ret);

Successfully decompressed logits!
Shape: torch.Size([512, 32100])
Data type: torch.float32
Min value: -12.0625
Max value: 39.4688
Sample values (first 5): [11.985416412353516, 29.16250228881836, 1.6791677474975586, -1.7562494277954102, -1.7562494277954102]
Compression format: quantized_4bit
Compression ratio: 59.81x
Original size: 31.35 MB
Compressed size: 0.52 MB


Let's start with understanding the data format. We have the /Data/dataset_with_predictions.jsonl file, containing the data (both input and output) for the teacher model.

In [2]:
NUM_LINES_TO_INSPECT = 5
DATA_PATH = "Data/dataset_with_predictions.jsonl"

inspected_data = []

with open(DATA_PATH, 'r') as data_file:
    for i, line_content in enumerate(data_file):
        if i >= NUM_LINES_TO_INSPECT:
            break
        data = json.loads(line_content.strip())
        inspected_data.append(data)

Now let's look closer at the parsed JSON entry:

In [3]:
print(inspected_data[0].keys())
print(inspected_data[0]["test_method_masked"])
print(inspected_data[0]["assertions"])
print(inspected_data[0]["teacher_prediction"])
print(inspected_data[0]["teacher_parsed_assertions"])
print(inspected_data[0]["teacher_metrics"])

dict_keys(['repository', 'focal_file', 'test_method_original', 'test_method_masked', 'assertions', 'method_under_test', 'teacher_prediction', 'teacher_parsed_assertions', 'teacher_metrics', 'teacher_logits'])
@Test
    public void testNaturalNumber() throws Exception {
        Object ret = reader.read("123");

    }
['assertNotNull(ret);', 'assertEquals(Long.class, ret.getClass());', 'assertEquals(123L, ret);']
assertNotNull(ret);
assertEquals(Integer.valueOf(3), ret);
['assertNotNull(ret);', 'assertEquals(Integer.valueOf(3), ret);']
{'precision': 1.0, 'recall': 0.6666666666666666, 'f1': 0.8, 'accuracy': 0.6666666666666666, 'similarity': 1.0, 'exact_matches': 2, 'generated_count': 2, 'reference_count': 3}


As we can see, the data contains the repository from which the data is taken, the file that contains the class that is being tested, the test method that was written, as well as a separated version of it (a masked test and the assertions separately), the method from the original class that is being tested, the prediction of the teacher model (both as a string and as a list of assertions), some teacher metrics regarding its prediction performance and the teacher's output logits (which we will use for the loss function of the student model).

Now, for every entry from the dataset, we need to construct an input for the student model that follows the same format as the input for the teacher model (as defined in DataGeneration/train_teacher_model.py). We also need to tokenize those inputs. We do this using the StudentDataset class, that will manage and tokenize the student model's input data:

In [5]:
# class StudentDataset(Dataset):
# This class was moved to Data/student_dataset for multithreading purposes

Now that we have the dataset class itself, we will also need a method to load the data that we have:

In [6]:
def load_dataset(jsonl_path, max_samples):
    """Load data from JSONL file"""
    data = []
    counter = 0
    with open(jsonl_path, 'r') as f:
        for line in f:
            if counter >= max_samples: break
            if line.strip():
                try:
                    data.append(json.loads(line))
                    counter += 1
                except json.JSONDecodeError:
                    continue
    return data

We also need a method to train the student model:

In [7]:
def train_student_model(model, tokenizer, train_dataloader, val_dataloader, args):
    """Train the student model to match the teacher's output on the assertion generation task"""

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Setup tensorboard if available
    try:
        from torch.utils.tensorboard import SummaryWriter
        tensorboard_writer = SummaryWriter(log_dir=os.path.join(args["output_dir"], "tensorboard"))
        use_tensorboard = True
    except ImportError:
        use_tensorboard = False

    # Prepare optimizer and scheduler
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args["weight_decay"],
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args["learning_rate"])

    # Calculate total training steps
    if args["max_steps"] > 0:
        t_total = args["max_steps"]
        num_epochs = args["max_steps"] // (len(train_dataloader) // args["gradient_accumulation_steps"]) + 1
    else:
        t_total = len(train_dataloader) // args["gradient_accumulation_steps"] * args["epochs"]
        num_epochs = args["epochs"]

    # Create scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=args["warmup_steps"],
        num_training_steps=t_total
    )

    # Mixed precision training if requested
    scaler = torch.cuda.amp.GradScaler() if args["fp16"] else None

    # Track metrics
    best_val_loss = float('inf')
    global_step = 0
    epochs_without_improvement = 0

    # For per-batch metrics
    batch_loss_window_size = args["batch_metrics_window"]
    batch_losses = []
    batch_accuracies = []
    batch_similarities = []
    eval_pool_size = args["batch_eval_pool"]  # Number of examples to evaluate in each batch

    # Create output directory
    os.makedirs(args["output_dir"], exist_ok=True)

    # Save training arguments
    with open(os.path.join(args["output_dir"], "training_args.json"), "w") as f:
        json.dump(args, f, indent=4)

    # Create metrics file
    metrics_file = os.path.join(args["output_dir"], "training_metrics.csv")
    with open(metrics_file, "w") as f:
        f.write("epoch,batch,global_step,loss,accuracy,similarity,lr,examples_per_second\n")

    # Main training loop
    print(f"Starting training for {num_epochs} epochs...")
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        epoch_start_time = time.time()
        examples_processed = 0

        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}")

        for batch_idx, batch in enumerate(progress_bar):
            batch_start_time = time.time()
            examples_in_batch = len(batch["input_ids"])
            examples_processed += examples_in_batch

            # Move batch to device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            teacher_logits = batch["teacher_logits"].to(device)

            # Forward pass with optional mixed precision
            if args["fp16"]:
                with torch.cuda.amp.autocast():
                    outputs = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels
                    )
                    loss = outputs.loss / args["gradient_accumulation_steps"]
                    student_pred_logits = student_outputs.logits
                    
                    if student_pred_logits.shape[1] != teacher_logits_batch.shape[1]:
                        print("Student and teacher logits sizes do not match")
                        pass
                    
                    active_loss_mask = student_labels.view(-1) != -100 # Flattened mask

                    active_student_log_probs = torch.nn.functional.log_softmax(
                        student_logits.view(-1, student_pred_logits.size(-1))[active_loss_mask] / args["distillation_temp"],
                        dim=-1
                    )
                    active_teacher_probs = torch.nn.functional.softmax(
                        teacher_logits.view(-1, teacher_logits_batch.size(-1))[active_loss_mask] / args["distillation_temp"],
                        dim=-1
                    )

                    if active_student_log_probs.numel() > 0:
                        loss_fct_kl = torch.nn.KLDivLoss(reduction="batchmean")
                        loss_distill = loss_fct_kl(
                            active_student_log_probs,
                            active_teacher_probs
                        ) * (args["distillation_temp"] ** 2) # Scale by T^2
                    else:
                        loss_distill = torch.tensor(0.0, device=loss_ce.device, dtype=loss_ce.dtype)
                        
                    loss = args["alpha_ce"] * loss + args["alpha_distil"] * loss_distill

                # Backward pass with gradient scaling
                scaler.scale(loss).backward()

                if (batch_idx + 1) % args["gradient_accumulation_steps"] == 0:
                    # Unscales the gradients
                    scaler.unscale_(optimizer)

                    # Clip gradients
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args["max_grad_norm"])

                    # Update weights
                    scaler.step(optimizer)
                    scheduler.step()
                    scaler.update()
                    optimizer.zero_grad()
                    global_step += 1
            else:
                # Standard forward and backward pass
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
                loss = outputs.loss / args["gradient_accumulation_steps"]
                student_pred_logits = student_outputs.logits
                    
                if student_pred_logits.shape[1] != teacher_logits_batch.shape[1]:
                    print("Student and teacher logits sizes do not match")
                    pass

                active_loss_mask = student_labels.view(-1) != -100 # Flattened mask

                active_student_log_probs = torch.nn.functional.log_softmax(
                    student_logits.view(-1, student_pred_logits.size(-1))[active_loss_mask] / args["distillation_temp"],
                    dim=-1
                )
                active_teacher_probs = torch.nn.functional.softmax(
                    teacher_logits.view(-1, teacher_logits_batch.size(-1))[active_loss_mask] / args["distillation_temp"],
                    dim=-1
                )

                if active_student_log_probs.numel() > 0:
                    loss_fct_kl = torch.nn.KLDivLoss(reduction="batchmean")
                    loss_distill = loss_fct_kl(
                        active_student_log_probs,
                        active_teacher_probs
                    ) * (args["distillation_temp"] ** 2) # Scale by T^2
                else:
                    loss_distill = torch.tensor(0.0, device=loss_ce.device, dtype=loss_ce.dtype)
                    
                loss = args["alpha_ce"] * loss + args["alpha_distil"] * loss_distill

                # Backward pass
                loss.backward()

                if (batch_idx + 1) % args["gradient_accumulation_steps"] == 0:
                    # Clip gradients
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args["max_grad_norm"])

                    # Update weights
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    global_step += 1

            # Track the loss
            loss_value = loss.item() * args["gradient_accumulation_steps"]
            epoch_loss += loss_value
            batch_losses.append(loss_value)
            if len(batch_losses) > batch_loss_window_size:
                batch_losses.pop(0)

            # Calculate time per example
            batch_time = time.time() - batch_start_time
            examples_per_second = examples_in_batch / batch_time if batch_time > 0 else 0

            # Per-batch metrics: Generate predictions for a few examples to calculate accuracy
            if args["track_batch_metrics"] and batch_idx % args["batch_metrics_every"] == 0:
                # Sample some examples from batch to evaluate
                eval_indices = np.random.choice(
                    range(len(input_ids)),
                    size=min(eval_pool_size, len(input_ids)),
                    replace=False
                )

                # Switch to eval mode temporarily
                model.eval()
                with torch.no_grad():
                    # Generate predictions for sampled examples
                    sampled_input_ids = input_ids[eval_indices]
                    sampled_attention_mask = attention_mask[eval_indices]

                    generated_ids = model.generate(
                        input_ids=sampled_input_ids,
                        attention_mask=sampled_attention_mask,
                        max_length=args["max_tgt_length"],
                        num_beams=4,
                        early_stopping=True
                    )

                    # Calculate accuracy and similarity
                    batch_accuracy = 0
                    batch_similarity = 0

                    for i, idx in enumerate(eval_indices):
                        generated_text = tokenizer.decode(generated_ids[i], skip_special_tokens=True)
                        reference_text = batch["original_target"][idx]

                        try:
                            metrics = evaluate_assertions(generated_text, reference_text)
                            batch_accuracy += metrics["accuracy"]
                            batch_similarity += metrics["similarity_score_avg"]
                        except Exception as e:
                            print(f"Error evaluating assertion: {e}")

                    # Average the metrics
                    batch_accuracy /= len(eval_indices) if eval_indices else 1
                    batch_similarity /= len(eval_indices) if eval_indices else 1

                # Switch back to train mode
                model.train()

                # Track metrics
                batch_accuracies.append(batch_accuracy)
                batch_similarities.append(batch_similarity)
                if len(batch_accuracies) > batch_loss_window_size:
                    batch_accuracies.pop(0)
                if len(batch_similarities) > batch_loss_window_size:
                    batch_similarities.pop(0)

                # Calculate moving averages
                avg_loss = sum(batch_losses) / len(batch_losses)
                avg_accuracy = sum(batch_accuracies) / len(batch_accuracies) if batch_accuracies else 0
                avg_similarity = sum(batch_similarities) / len(batch_similarities) if batch_similarities else 0

                # Update progress bar
                progress_bar.set_postfix({
                    "loss": avg_loss,
                    "accuracy": avg_accuracy,
                    "similarity": avg_similarity,
                    "ex/s": f"{examples_per_second:.1f}"
                })

                # Log to tensorboard
                if use_tensorboard:
                    tensorboard_writer.add_scalar("batch_loss", avg_loss, global_step)
                    tensorboard_writer.add_scalar("batch_accuracy", avg_accuracy, global_step)
                    tensorboard_writer.add_scalar("batch_similarity", avg_similarity, global_step)
                    tensorboard_writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step)
                    tensorboard_writer.add_scalar("examples_per_second", examples_per_second, global_step)

                # Log to CSV
                with open(metrics_file, "a") as f:
                    f.write(
                        f"{epoch + 1},{batch_idx + 1},{global_step},{avg_loss:.6f},{avg_accuracy:.6f},{avg_similarity:.6f},{scheduler.get_last_lr()[0]:.8f},{examples_per_second:.2f}\n")
            else:
                # Just update with loss
                avg_loss = sum(batch_losses) / len(batch_losses)
                progress_bar.set_postfix({
                    "loss": avg_loss,
                    "ex/s": f"{examples_per_second:.1f}"
                })

            # Evaluate periodically
            if args["eval_steps"] > 0 and global_step % args["eval_steps"] == 0:
                val_loss, eval_results = evaluate_model(model, tokenizer, val_dataloader, device)

                # Log to tensorboard
                if use_tensorboard:
                    tensorboard_writer.add_scalar("eval_loss", val_loss, global_step)
                    for metric, value in eval_results.items():
                        if isinstance(value, (int, float)):
                            tensorboard_writer.add_scalar(f"eval_{metric}", value, global_step)

                # Print evaluation results
                print(f"\nEvaluation at step {global_step}:")
                print(f"  Validation loss: {val_loss:.4f}")
                print(f"  Similarity score: {eval_results['similarity_score_avg']:.4f}")
                print(f"  Accuracy: {eval_results['accuracy']:.4f}")
                print(f"  F1 score: {eval_results['f1']:.4f}")

                # Save best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    print(f"  New best validation loss: {val_loss:.4f}")

                    # Save model checkpoint
                    model_dir = os.path.join(args["output_dir"], "best_model")
                    os.makedirs(model_dir, exist_ok=True)
                    model.save_pretrained(model_dir)
                    tokenizer.save_pretrained(model_dir)

                    # Save optimizer and scheduler
                    torch.save(optimizer.state_dict(), os.path.join(model_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(model_dir, "scheduler.pt"))

                    # Reset patience counter
                    epochs_without_improvement = 0
                else:
                    # Increment patience counter
                    epochs_without_improvement += 1

                # Early stopping
                if 0 < args["early_stopping_patience"] <= epochs_without_improvement:
                    print(f"Early stopping after {epochs_without_improvement} evaluations without improvement")
                    break

                # Back to training mode
                model.train()

            # Save checkpoint
            if args["save_steps"] > 0 and global_step % args["save_steps"] == 0:
                checkpoint_dir = os.path.join(args["output_dir"], f"checkpoint-{global_step}")
                os.makedirs(checkpoint_dir, exist_ok=True)
                model.save_pretrained(checkpoint_dir)
                tokenizer.save_pretrained(checkpoint_dir)

                # Save optimizer and scheduler
                torch.save(optimizer.state_dict(), os.path.join(checkpoint_dir, "optimizer.pt"))
                torch.save(scheduler.state_dict(), os.path.join(checkpoint_dir, "scheduler.pt"))

            # Break if max steps reached
            if args["max_steps"] > 0 and global_step >= args["max_steps"]:
                break

        # Calculate average loss for the epoch
        epoch_avg_loss = epoch_loss / len(train_dataloader)
        train_losses.append(epoch_avg_loss)

        # Calculate epoch time and speed
        epoch_time = time.time() - epoch_start_time
        examples_per_second = examples_processed / epoch_time if epoch_time > 0 else 0

        print(f"\nEpoch {epoch + 1}/{num_epochs} completed in {epoch_time:.2f}s ({examples_per_second:.2f} examples/s)")
        print(f"  Average training loss: {epoch_avg_loss:.4f}")

        # Evaluate at the end of each epoch
        print(f"  Evaluating epoch {epoch + 1}...")
        val_loss, eval_results = evaluate_model(model, tokenizer, val_dataloader, device)
        val_losses.append(val_loss)

        # Log to tensorboard
        if use_tensorboard:
            tensorboard_writer.add_scalar("epoch_train_loss", epoch_avg_loss, epoch + 1)
            tensorboard_writer.add_scalar("epoch_val_loss", val_loss, epoch + 1)
            for metric, value in eval_results.items():
                if isinstance(value, (int, float)):
                    tensorboard_writer.add_scalar(f"epoch_eval_{metric}", value, epoch + 1)

        # Print evaluation results
        print(f"  Validation loss: {val_loss:.4f}")
        print(f"  Similarity score: {eval_results['similarity_score_avg']:.4f}")
        print(f"  Accuracy: {eval_results['accuracy']:.4f}")
        print(f"  F1 score: {eval_results['f1']:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print(f"  New best validation loss: {val_loss:.4f}")

            # Save model checkpoint
            model_dir = os.path.join(args["output_dir"], "best_model")
            os.makedirs(model_dir, exist_ok=True)
            model.save_pretrained(model_dir)
            tokenizer.save_pretrained(model_dir)

            # Reset patience counter
            epochs_without_improvement = 0
        else:
            # Increment patience counter
            epochs_without_improvement += 1

        # Early stopping
        if 0 < args["early_stopping_patience"] <= epochs_without_improvement:
            print(f"Early stopping after {epochs_without_improvement} epochs without improvement")
            break

    # Save final model
    final_model_dir = os.path.join(args["output_dir"], "final_model")
    os.makedirs(final_model_dir, exist_ok=True)
    model.save_pretrained(final_model_dir)
    tokenizer.save_pretrained(final_model_dir)

    # Close tensorboard writer
    if use_tensorboard:
        tensorboard_writer.close()

    # Plot loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(os.path.join(args["output_dir"], "loss_curves.png"))
    plt.close()

    return model, tokenizer, best_val_loss

Now we will also need to run the training of our student model. For this, we will need to define the training configuration and a method to run the training with that configuration:

In [8]:
config = {
    # --- Data Args ---
    "data_path": "Data/dataset_with_predictions.jsonl",
    "output_dir": "./student_model_output",
    "model_name": "Salesforce/codet5-small",
    "validation_split": 0.1,
    "max_src_length": 1024,
    "max_tgt_length": 512,

    # --- Distillation Args ---
    "distillation_temp": 2.0,
    "alpha_ce": 0.0,            # Weight for student's own Cross-Entropy loss
    "alpha_distil": 1.0,        # Weight for distillation loss (e.g., KL divergence)

    # --- Training Args ---
    "epochs": 3,
    "batch_size": 8,
    "eval_batch_size": 16,
    "gradient_accumulation_steps": 1,
    "learning_rate": 5e-5,
    "weight_decay": 0.01,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "max_steps": -1,
    "fp16": True,
    "seed": 42,

    # --- Batch Metrics Args (Optional, can simplify by removing) ---
    "track_batch_metrics": False,
    "batch_metrics_every": 50,
    "batch_metrics_window": 50,
    "batch_eval_pool": 4,

    # --- Logging and Saving Args ---
    "logging_steps": 100,
    "eval_strategy": "epoch",
    "eval_steps": 500,
    "save_strategy": "epoch",
    "save_steps": 1000,
    "save_total_limit": 2,
    "early_stopping_patience": 3,
    "num_workers": 2,           
    "max_samples": 100,
}

In [9]:
def run_student_training(args):
    # Set random seed
    torch.manual_seed(args["seed"])
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args["seed"])

    # Load dataset
    print(f"Loading dataset from {args['data_path']}...")
    if args["max_samples"] is not None:
        data = load_dataset(args["data_path"], args["max_samples"])
        print(f"Using first {len(data)} examples")
    else:
        data = load_dataset(args["data_path"])
        print(f"Loaded {len(data)} examples")

    # Split into train and validation sets
    train_data, val_data = train_test_split(data, test_size=args["validation_split"], random_state=args["seed"])
    print(f"Training on {len(train_data)} examples, validating on {len(val_data)} examples")

    # Load tokenizer and model
    print(f"Loading model: {args['model_name']}")
    tokenizer = RobertaTokenizer.from_pretrained(args["model_name"])
    model = T5ForConditionalGeneration.from_pretrained(args["model_name"])

    # Create datasets
    train_dataset = StudentDataset(
        train_data,
        tokenizer,
        max_src_length=args["max_src_length"],
        max_tgt_length=args["max_tgt_length"]
    )
    val_dataset = StudentDataset(
        val_data,
        tokenizer,
        max_src_length=args["max_src_length"],
        max_tgt_length=args["max_tgt_length"]
    )

    # Create dataloaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=args["batch_size"],
        shuffle=True,
        num_workers=args["num_workers"],
        pin_memory=True
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=args["eval_batch_size"],
        shuffle=False,
        num_workers=args["num_workers"],
        pin_memory=True
    )

    # Check for CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Train model
    model, tokenizer, best_val_loss = train_student_model(
        model,
        tokenizer,
        train_dataloader,
        val_dataloader,
        args
    )

    print(f"Training completed! Best validation loss: {best_val_loss:.4f}")
    print(f"Trained model and checkpoints saved to {args['output_dir']}")

In [10]:
run_student_training(config)

Loading dataset from Data/dataset_with_predictions.jsonl...
Using first 100 examples
Training on 90 examples, validating on 10 examples
Loading model: Salesforce/codet5-small


D:\Delft\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using device: cuda


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5480\2635291341.py:45: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if args["fp16"] else None


Starting training for 3 epochs...


Epoch 1/3:   0%|                                                                        | 0/12 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5480\2635291341.py:97: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/3:   0%|                                                                        | 0/12 [00:27<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 252.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 8.18 GiB is allocated by PyTorch, and 58.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)